## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [12]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how}게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

In [13]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how}게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002333ABEBD10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002333AC12150>, root_client=<openai.OpenAI object at 0x000002333AAF8290>, root_async_client=<openai.AsyncOpenAI object at 0x000002333ABFAA10>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [18]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리",  "how" : "영어로"}

In [19]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='The learning principle of artificial intelligence (AI) models primarily revolves around the concept of machine learning, which enables systems to learn from data and improve their performance over time without being explicitly programmed for every task. Here’s a brief overview of the key components involved in this process:\n\n1. **Data Collection**: AI models require large amounts of data to learn from. This data can be in various forms, such as images, text, or numerical values, and it is essential for training the model.\n\n2. **Feature Extraction**: In this step, relevant features or characteristics are identified from the raw data. This process helps the model focus on the most important aspects of the data that contribute to making predictions or decisions.\n\n3. **Model Selection**: Different algorithms can be used to create AI models, such as decision trees, neural networks, or support vector machines. The choice of model depends on the nature of the data and

아래는 스트리밍을 출력하는 예시 입니다.

In [20]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

The learning principle of artificial intelligence (AI) models primarily revolves around the concept of machine learning, which enables systems to learn from data and improve their performance over time without being explicitly programmed for every task. Here’s a breakdown of the key components involved in this process:

1. **Data Collection**: AI models require large amounts of data to learn from. This data can be in various forms, such as text, images, audio, or numerical values. The quality and quantity of the data significantly influence the model's performance.

2. **Feature Extraction**: In many cases, raw data needs to be transformed into a format that the model can understand. This involves identifying and extracting relevant features or characteristics from the data that will help the model make predictions or decisions.

3. **Model Selection**: There are various types of machine learning models, such as supervised learning, unsupervised learning, and reinforcement learning. Th

### 출력파서(Output Parser)


In [21]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [22]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [23]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how}게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002333ABEBD10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002333AC12150>, root_client=<openai.OpenAI object at 0x000002333AAF8290>, root_async_client=<openai.AsyncOpenAI object at 0x000002333ABFAA10>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [24]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [25]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [26]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  1. Customer: "Hi, I would like to see the menu, please."
  2. Waiter: "Of course! Here you go. Can I get you something to drink while you decide?"
  3. Customer: "Yes, I would like a glass of water, please."
  4. Waiter: "Sure! Are you ready to order, or do you need more time?"
  5. Customer: "I think I'm ready. I would like the grilled chicken with a side salad."
  6. Waiter: "Great choice! Would you like any dressing with your salad?"
  7. Customer: "Yes, please. I'll have the balsamic vinaigrette."
  8. Waiter: "Perfect! Your order will be ready shortly."

- 한글 해석:
  1. 손님: "안녕하세요, 메뉴를 보고 싶어요."
  2. 웨이터: "물론이죠! 여기 있습니다. 결정하시는 동안 음료수 드릴까요?"
  3. 손님: "네, 물 한 잔 주세요."
  4. 웨이터: "알겠습니다! 주문할 준비가 되셨나요, 아니면 더 필요하신가요?"
  5. 손님: "이제 준비된 것 같아요. 그릴에 구운 치킨과 사이드 샐러드를 주문할게요."
  6. 웨이터: "좋은 선택입니다! 샐러드에 드레싱을 추가할까요?"
  7. 손님: "네, 발사믹 비네그렛 드레싱으로 주세요."
  8. 웨이터: "완벽합니다! 주문하신 음식은 곧 준비될 거예요."


In [27]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  1. Customer: "Hi, could I see the menu, please?"
  2. Waiter: "Of course! Here you go."
  3. Customer: "Thank you! I think I'll have the grilled salmon."
  4. Waiter: "Great choice! Would you like any sides with that?"
  5. Customer: "Yes, I'll have a side of steamed vegetables, please."
  6. Waiter: "Perfect! Anything to drink?"
  7. Customer: "I'll have a glass of water, please."

- 한글 해석:
  1. 손님: "안녕하세요, 메뉴 좀 보여주실 수 있나요?"
  2. 웨이터: "물론입니다! 여기 있습니다."
  3. 손님: "감사합니다! 저는 구운 연어를 주문할게요."
  4. 웨이터: "좋은 선택입니다! 사이드 메뉴는 필요하신가요?"
  5. 손님: "네, 찐 야채 사이드를 주세요."
  6. 웨이터: "완벽합니다! 음료는 무엇을 드릴까요?"
  7. 손님: "물 한 잔 주세요."

In [28]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
Customer: Hi, I’d like to order a pizza, please.  
Pizza Shop: Sure! What size would you like?  
Customer: I’ll have a large pizza, please.  
Pizza Shop: Great! What toppings do you want?  
Customer: Can I get pepperoni, mushrooms, and extra cheese?  
Pizza Shop: Absolutely! Would you like anything to drink?  
Customer: Yes, a 2-liter bottle of cola, please.  
Pizza Shop: Perfect! Your total comes to $25. Would you like it for delivery or pick-up?  
Customer: I’ll pick it up. How long will it take?  
Pizza Shop: It should be ready in about 20 minutes.  
Customer: Thank you! See you soon.  
Pizza Shop: You’re welcome! See you!

- 한글 해석:
고객: 안녕하세요, 피자 하나 주문하고 싶어요.  
피자 가게: 물론이죠! 어떤 사이즈로 드릴까요?  
고객: 큰 피자로 해주세요.  
피자 가게: 좋습니다! 어떤 토핑을 원하시나요?  
고객: 페퍼로니, 버섯, 그리고 치즈를 추가해 주세요.  
피자 가게: 알겠습니다! 음료수는 필요하신가요?  
고객: 네, 2리터 콜라 하나 주세요.  
피자 가게: 완벽해요! 총 금액은 25달러입니다. 배달로 하실 건가요, 아니면 픽업으로 하실 건가요?  
고객: 저는 픽업할게요. 얼마나 걸릴까요?  
피자 가게: 약 20분 정도 소요될 것입니다.  
고객: 감사합니다! 곧 뵙겠습니다.  
피자 가게: 천만에요! 곧 뵙겠습니다!